In [ ]:
import numpy as np
import xarray as xr
import hvplot.xarray
import cartopy.crs as crs, geoviews as gv
import holoviews as hv
import glob

gv.extension('bokeh', logo=False)
hv.extension('bokeh', logo=False)

In [ ]:
outcrs = crs.epsg(6676)
url = 'https://cyberjapandata.gsi.go.jp/xyz/seamlessphoto/{Z}/{X}/{Y}.jpg'
geomap = gv.WMTS(url, crs=outcrs) #.options(width=600, height=400)

In [ ]:
inputfile = 'out00000040.nc'
outputfile = '10hour.html'

In [ ]:
ds = xr.open_dataset(inputfile)

In [ ]:
ds

In [ ]:
ds.depth.values[ ds.depth.values < 0.01 ] = np.nan

In [ ]:
out1 = ds.hvplot.quadmesh( 'xc','yc',z='depth'
                          , crs=outcrs, rasterize=True, dynamic=False, cmap='jet'
                          ,width=600, height=600,colorbar=True ) #.redim.range(depth=(0.2,1.5))

In [ ]:
x0 = ds.xc.values[0,0]
x1 = ds.xc.values[1,0]
y0 = ds.yc.values[0,0]
y1 = ds.yc.values[1,0]
vc = [x1-x0, y1-y0]
th = np.arccos( vc[0]/np.linalg.norm(vc) ) 

U =  ds.u.values[::3,::3]
V =  ds.v.values[::3,::3]
X = ds.xc.values[::3,::3]
Y = ds.yc.values[::3,::3]

mag = np.sqrt(U**2 + V**2) + 0.00000001 
angle = (np.pi/2.) - np.arctan2(U/mag, V/mag) + th
vectorfield = gv.VectorField((X, Y, angle, mag), crs=outcrs).options(magnitude='Magnitude', scale=0.5, color='white')

In [ ]:
g = out1.opts(alpha=0.5) * vectorfield * geomap

In [ ]:
g

In [ ]:
hv.save(g,outputfile)